In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)
italian_ingredients = list(set(italian_df.ingredients))

mexican_df = DataFrame(clean_mexican_cuisine_data)
mexican_ingredients = list(set(mexican_df.ingredients))


In [6]:
#fixing data for CountVectorizer (italian_df)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)
      

In [7]:
#fixing data for CountVectorizer (mexican_df)

mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)

In [8]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')
#print(it_df)

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')
#print(mex_df)

In [9]:
#putting mexican and italian in one DataFrame

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

In [10]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [11]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)
mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [12]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [13]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')
#print(it2_df)

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')
#print(mex2_df)

In [14]:
#putting mexican and italian in one DataFrame

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)
print(df2)

       abbamele   ac  accent  acini  acorn  active  added  adobo  agave  aged  \
4           0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
14          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
20          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
56          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
62          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
...         ...  ...     ...    ...    ...     ...    ...    ...    ...   ...   
49678       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49687       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49707       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49709       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49717       0.0  0.0     0.0    0.0    0.0     0.0    0.0    1.0    0.0   0.0   

       ...  wensleydale  we

In [15]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.SVC()

#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = BernoulliNB(binarize = None)

In [22]:
print('********* CountVectorizer *********')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

********* CountVectorizer *********
(14276, 3976)


In [17]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

****Logistic Regression mean scores****
scores: {'fit_time': array([2.31020904, 2.0499053 , 1.96741199, 1.95003009, 2.04533005]), 'score_time': array([0.13899088, 0.13435984, 0.10122919, 0.1008358 , 0.127141  ]), 'test_accuracy': array([0.96813725, 0.97233894, 0.96778711, 0.96671338, 0.9688157 ]), 'test_precision_weighted': array([0.96837424, 0.97241883, 0.96788602, 0.96682014, 0.96883557]), 'test_recall_weighted': array([0.96813725, 0.97233894, 0.96778711, 0.96671338, 0.9688157 ]), 'test_f1_weighted': array([0.96809087, 0.97231503, 0.96775533, 0.96667911, 0.96879857])}

mean score test accuracy: 0.9687584774624636

mean score test precision weighted: 0.9688669615432207

mean score test recall weighted: 0.9687584774624636

mean score test f1-measure weighted: 0.968727783410316


/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

Confusion Matrix: [[7679  159]
 [ 287 6151]]


In [ ]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

In [23]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****d-tree mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****d-tree mean scores****
scores: {'fit_time': array([29.54377818, 24.38601685, 28.68981385, 28.7813518 , 20.56070304]), 'score_time': array([0.10330701, 0.09680319, 0.08734488, 0.08625317, 0.08562613]), 'test_accuracy': array([0.93767507, 0.95133053, 0.93872549, 0.94253679, 0.93062369]), 'test_precision_weighted': array([0.93766729, 0.95138607, 0.93875214, 0.9425827 , 0.93080495]), 'test_recall_weighted': array([0.93767507, 0.95133053, 0.93872549, 0.94253679, 0.93062369]), 'test_f1_weighted': array([0.93767072, 0.95128451, 0.93873592, 0.94255228, 0.93066835])}

mean score test accuracy: 0.9401783137922303

mean score test precision weighted: 0.9402386303037866

mean score test recall weighted: 0.9401783137922303

mean score test f1-measure weighted: 0.9401823547873281
Confusion Matrix: [[7412  426]
 [ 436 6002]]


In [ ]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

In [ ]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

In [18]:
print('********* TfidfVectorizer *********')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

********* TfidfVectorizer *********
(14276, 2006)


In [19]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to '

****Logistic Regression mean scores****
scores: {'fit_time': array([0.8786943 , 0.82599497, 0.81366682, 0.82805109, 0.800915  ]), 'score_time': array([0.02411008, 0.02425408, 0.02461004, 0.02333093, 0.02349782]), 'test_accuracy': array([0.99194678, 0.9929972 , 0.9929972 , 0.99544499, 0.99404345]), 'test_precision_weighted': array([0.9920632 , 0.9930854 , 0.9930854 , 0.99548247, 0.99410738]), 'test_recall_weighted': array([0.99194678, 0.9929972 , 0.9929972 , 0.99544499, 0.99404345]), 'test_f1_weighted': array([0.99193982, 0.99299199, 0.99299199, 0.99544284, 0.99403972])}

mean score test accuracy: 0.9934859227503194

mean score test precision weighted: 0.9935647664446428

mean score test recall weighted: 0.9934859227503194

mean score test f1-measure weighted: 0.9934812704260404


/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Confusion Matrix: [[7838    0]
 [  93 6345]]


/Users/jo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

In [20]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****d-tree mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****d-tree mean scores****
scores: {'fit_time': array([4.13364887, 4.21988606, 3.97078705, 3.98422503, 4.41084409]), 'score_time': array([0.051373  , 0.04051805, 0.04074407, 0.04045296, 0.0480361 ]), 'test_accuracy': array([0.99754902, 0.99754902, 0.9982493 , 0.99824807, 0.99894884]), 'test_precision_weighted': array([0.99755991, 0.99755433, 0.99825486, 0.99825365, 0.99895085]), 'test_recall_weighted': array([0.99754902, 0.99754902, 0.9982493 , 0.99824807, 0.99894884]), 'test_f1_weighted': array([0.99754841, 0.99754859, 0.99824899, 0.99824776, 0.99894873])}

mean score test accuracy: 0.9981088511087688

mean score test precision weighted: 0.9981147215558022

mean score test recall weighted: 0.9981088511087688

mean score test f1-measure weighted: 0.9981084968101144
Confusion Matrix: [[7837    1]
 [  24 6414]]


In [ ]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

In [ ]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)